In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 32601343
paper_name = 'baruffini_lodi_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [6]:
original_data = pd.read_excel('raw_data/41598_2020_67439_MOESM2_ESM-2.xls', sheet_name='Suppl_Table S1', skiprows=2)

In [7]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 327 x 5


In [8]:
original_data.head()

,Systematic Name,Standard Name,Phenotype1,Description,Feature Type
0,YDR127W,ARO1,HS,"Pentafunctional arom protein, catalyzes steps ...",ORF
1,YGL148W,ARO2,HS,Bifunctional chorismate synthase and flavin re...,ORF
2,YOR349W,CIN1,HS,Tubulin folding factor D involved in beta-tubu...,ORF
3,YMR202W,ERG2,HS,"C-8 sterol isomerase, catalyzes the isomerizat...",ORF
4,YNL280C,ERG24,HS,"C-14 sterol reductase, acts in ergosterol bios...",ORF


In [9]:
original_data['orf'] = original_data['Systematic Name'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                               Systematic Name Standard Name  \
index_input                                                                    
325                                                        NaN           NaN   
326          1Phenotypes observed: HS, high sensitive mutan...           NaN   

            Phenotype1 Description Feature Type  \
index_input                                       
325                NaN         NaN          NaN   
326                NaN         NaN          NaN   

                                                           orf  
index_input                                                     
325                                                        NAN  
326          1PHENOTYPESOBSERVEDHSHIGHSENSITIVEMUTANTMSMEDI...  


In [13]:
original_data = original_data.loc[t,:]

In [14]:
data_dict = {'HS': -3, 'MS': -2, 'LS': -1, 'R': 1}
original_data['data'] = original_data['Phenotype1'].apply(lambda x: data_dict[x])

In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(325, 1)

# Load & process tested strains

In [19]:
tested = pd.read_excel('raw_data/yeast deletion mutant collection.xls', sheet_name='tot')

In [20]:
tested.head()

,Systematic Name,Bg strain,ORF name,Alias,Description,Systematic Name.1,Phenotype [Ni(S-tcitr)2-sensitive or -tolerant mutants]
0,YAL002W,BY4739,VPS8,VPL8|VPT8|FUN15,Membrane-associated protein that interacts wit...,YAL002W,NaN
1,YAL004W,BY4739,NaN,NaN,Dubious open reading frame unlikely to encode ...,YAL004W,NaN
2,YAL005C,BY4739,SSA1,YG100,ATPase involved in protein folding and nuclear...,YAL005C,NaN
3,YAL007C,BY4739,ERP2,NaN,Protein that forms a heterotrimeric complex wi...,YAL007C,NaN
4,YAL008W,BY4739,FUN14,NaN,Mitochondrial protein of unknown function,YAL008W,NaN


In [21]:
tested['orf'] = tested['Systematic Name'].astype(str)

In [22]:
tested['orf'] = clean_orf(tested['orf'])

In [23]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [24]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [Systematic Name, Bg strain, ORF name, Alias, Description, Systematic Name.1, Phenotype [Ni(S-tcitr)2-sensitive or -tolerant mutants], orf]
Index: []


In [25]:
tested_orfs = tested['orf'].unique()

In [26]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [27]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [28]:
original_data.shape

(4677, 1)

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [21862]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,21862
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21862
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      21862          
data_type       value    valuez
gene_id orf                    
2       YAL002W     0  0.021556
1863    YAL004W     0  0.021556
4       YAL005C     0  0.021556
5       YAL007C     0  0.021556
6       YAL008W     0  0.021556

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 32601343...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.07s/it]

Updating the data_modified_on field...
